# METU Technology and Regulation II - Big Data
This material was made for the Technology and regulation II. course of METU. Contact debreczenim@mnb.hu for permission to use.

## What is "Big Data"?

A buzzword.

Lots of definitions online, look them up if you are interested.
However, our current digital economy runs on data (lots of it).

Consider social media *in 2017*:

- Snapchat users share 527,760 photos
- Users watch 4,146,600 YouTube videos
- 456,000 tweets are sent on Twitter
- Instagram users post 46,740 photos

### on average, every minute of every day [[1](https://www.forbes.com/sites/bernardmarr/2018/05/21/how-much-data-do-we-create-every-day-the-mind-blowing-stats-everyone-should-read)]

**The amount of data generated and the speed of it's creation is ever increasing.**

## Why bother processing it?

- Profitable for the companies
- Targeted ads make content free for users
- Granular metrics can better support the decision-making processes

### Facebook's annual ad revenue in million dollars [[2](https://www.statista.com/statistics/271258/facebooks-advertising-revenue-worldwide/)]
<img src="pics/facebook-ad-revenue.png" alt="facebook annual ad revenue in million dollars" />

## How to process it?

As with everything in software: **it depends...**
- on the amount,
- the structure/format,
- the frequency,
- the sensitivity
- etc. 

of the data. Designing the pipelines and architecture best suited for each use-case is a complex task.

## Supporting hardware

- Vertical scaling of compute can not keep up with the rate of data creation
- Modern CPU-s are multi-core and can run multiple processes in parallel
- Modern large-scale data pipelines consist of multiple servers
- These require specialised software to utilize the performance gains

### CPU benchmark scores (Passmark) [[3](https://www.cpubenchmark.net/year-on-year.html)]
<img src="pics/cpu-performance.png" alt="cpu average performance throughout the years" />


### Cloud infrastructure

- Cloud is also used as a buzzword
- Oversimplified: Cloud = lots of computers available for renting 
- In realitiy: specialized servers + software + service etc.
- Virtual machines communicate with eachother to process data

Amazon's and Netflix's [microservices](https://en.wikipedia.org/wiki/Microservices) architecture
<img src="pics/microservices.webp" alt="amazon and netflix microservice architecture graph" />

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf
import dask.dataframe as dd
import pandas as pd

from colorcet import fire

## Pandas dataframes
Tabular data, similar to excel sheets. Makes common calculations easy, by providing a user frendly api.

**Works well on "small" data.**

In [ ]:
passing_dataset = {
  'cars': ["BMW", "Volvo", "Ford"],
  'passings': [3, 7, 2]
}

passing_df = pd.DataFrame(passing_dataset)
passing_df

In [ ]:
price_dataset = {
    'cars': ['Volvo', 'Ford', 'BMW', 'Hyundai'],
    'price':[1000, 2000, 3000, 4000]
}
price_df = pd.DataFrame(price_dataset)
price_df

In [ ]:
merged_df = pd.merge(price_df, passing_df, on='cars', how='outer')
merged_df

In [ ]:
merged_df['price_to_passing_ratio'] = merged_df['price'] / merged_df['passings']
merged_df

In [ ]:
merged_df.loc[merged_df['price_to_passing_ratio'] == merged_df['price_to_passing_ratio'].min()]

## Dask dataframes

Provides the same/similar API as pandas for handling tabular data.

Dask dataframes use pandas dataframes under the hood and operations on them are mapped to operations on the underlying pandas dataframes.
However, **the operations on the pandas dataframes are performed lazily and in parallel**, as dask is optimized for handling *large* amounts of data.

In [ ]:
df = pd.read_parquet('data/osm-1billion.parq')
df

In [ ]:
df = dd.read_parquet('data/osm-1billion.parq')
df

In [ ]:
df = df.mean()
df

In [ ]:
df.visualize()

In [ ]:
df.compute()

In [ ]:
bound = 20026376.39
bounds = dict(x_range = (-bound, bound), y_range = (int(-bound*0.4), int(bound*0.6)))
plot_width = 900
plot_height = int(plot_width*0.5)

In [ ]:
df = dd.read_parquet('data/osm-1billion.parq')
df

In [ ]:
%%time
cvs = ds.Canvas(plot_width=plot_width, plot_height=plot_height, **bounds)
agg = cvs.points(df, 'x', 'y', ds.count())

In [ ]:
tf.shade(agg, cmap=["lightblue", "darkblue"], how='log')

In [ ]:
tf.set_background(tf.shade(agg, cmap=fire), "black")